In [217]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.datasets import load_diabetes

## Load in the dataset into a DataFrame

In [218]:
df = load_diabetes(as_frame=True).frame

In [219]:
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


## Plot the median income to the median house value

In [220]:
def create_plot(x, y, x_line=None, y_line=None):
    # Create the scatter plot
    fig = px.scatter(x=x, y=y, title="Body Mass Index (BMI) correlation with Target")

    fig.update_layout(
        xaxis_title="Body Mass Index (BMI)",
        yaxis_title="Target"
    )

    # Add the best fit line if the values are provided
    if x_line is None or y_line is None:
        return fig
    
    fig.add_trace(
        go.Scatter(
            x=x_line,
            y=y_line,
            mode='lines',
            name='Best Fit Line',
            line=dict(color='red', width=2)
        )
    )
    return fig

In [221]:
x = df["bmi"]
y = df["target"]
fig = create_plot(x, y)
fig.show()

## To find the best fit line, we should start with a random slope and intercept

In [222]:
m = 1
b = 0

In [223]:
x_line = np.linspace(min(x), max(x), 10)
y_line = m * x_line + b
fig = create_plot(x, y, x_line, y_line)
fig.show()

## Calculate the errors for each prediction

In [224]:
def calculate_mse(_m, _b):
    errors = [((_m * _x + _b) - _y)for (_x, _y) in zip(x, y)]
    errors_squared = [e * e for e in errors]
    mse = np.average(errors_squared)
    return mse

## Compute the gradients and update the slop and intercept until the loss converges

In [225]:
alpha = 0.05
epsilon = 1e-8
max_iterations = 50000
N = len(x)

In [226]:
def gradient_step(_m, _b):
    new_errors = _m * x + _b - y
    grad_m = (2 / N) * np.dot(new_errors, x)
    grad_b = (2 / N) * sum(new_errors)

    new_m = _m - alpha * grad_m
    new_b = _b - alpha * grad_b
    return new_m, new_b

In [227]:
losses = []
prev_loss = calculate_mse(m, b)
loss = prev_loss + 10
while abs(prev_loss - loss) > epsilon and len(losses) < max_iterations:
    prev_loss = loss
    loss = calculate_mse(m, b)
    losses.append(loss)
    if abs(prev_loss - loss) <= epsilon:
        break
    m, b = gradient_step(m, b)

## Plot the loss

In [230]:
losses_fig = px.line(losses[20:], title="MSE Losses")
losses_fig.update_layout(
    xaxis_title="Index",
    yaxis_title="Loss"
)
losses_fig.show()

## Report the final results

In [229]:
print(f"Final Results:\n-> Best Fit Line:\ty = {m}x + {b}\n-> Final Loss Diff:\t{prev_loss - loss}")
x_line = np.linspace(min(x), max(x), 10)
y_line = m * x_line + b
fig = create_plot(x, y, x_line, y_line)
fig.show()

Final Results:
-> Best Fit Line:	y = 949.336449701473x + 152.13348416289602
-> Final Loss Diff:	9.998984751291573e-09
